In [1]:
import onnxruntime as ort

providers = ["CPUExecutionProvider"]
sess_options = ort.SessionOptions()
onnx_sess = ort.InferenceSession(
    "vits-en-AU-Dean2Zak.onnx",
    sess_options=sess_options,
    providers=providers,
)

In [2]:
import numpy as np

input_ids = [ 0, 28,  0, 50,  0,  7,  0, 47,  0,  7,  0, 25,  0, 50,  0,  7,  0, 58,
          0, 24,  0,  7,  0, 59,  0, 30,  0,  7,  0, 55,  0, 24,  0, 37,  0, 20,
          0,  7,  0,  5,  0]
input_lengths = np.array([len(input_ids)], dtype=np.int64)
input_ids = np.array([input_ids], dtype=np.int64)
noise_scale = np.array([.667], dtype=np.float32)
noise_scale_w = np.array([.8], dtype=np.float32)
length_scale = np.array([1.0], dtype=np.float32)

In [3]:
outputs = onnx_sess.run(
    None,
    {
        "input": input_ids,
        "input_lengths": input_lengths,
        "noise_scale": noise_scale,
        "noise_scale_w": noise_scale_w,
        "length_scale": length_scale,
        "speaker_id": None,
    },
)

In [4]:
audio, attention = outputs

In [5]:
import IPython.display as ipd

ipd.Audio(audio[0, 0, :], rate=44100)

In [6]:
audio.shape

(1, 1, 66048)

In [7]:
attention.shape

(1, 1, 129, 41)

In [8]:
duration_s = audio.shape[-1] / 44100
duration_s

1.497687074829932

In [9]:
durations = np.sum(attention[0][0], axis=0)
durations.shape, durations

((41,),
 array([5., 3., 3., 5., 5., 2., 2., 5., 4., 2., 3., 7., 1., 3., 2., 2., 2.,
        5., 1., 2., 1., 3., 2., 1., 2., 2., 1., 3., 4., 6., 3., 7., 2., 3.,
        3., 2., 6., 3., 7., 1., 3.], dtype=float32))

In [ ]:
# 1 acoustic unit = 256 frames = 256 frames / 22050 fps = 0,011609977324263 s